In [5]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np

In [7]:
### Load the trained model, scaler pickle,onehot
model=load_model('model.h5')

## load the encoder and scaler
with open('onehot_encoder_geo.pkl','rb') as file:
    label_encoder_geo=pickle.load(file)

with open('label_encoder_gender.pkl', 'rb') as file:
    label_encoder_gender = pickle.load(file)

with open('scaler.pkl', 'rb') as file:
    scaler = pickle.load(file)

In [ ]:
# Example input data
input_data = {
    'CreditScore': 600,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}

In [ ]:
from sklearn.preprocessing import OneHotEncoder
import pandas as pd

# Example input
input_data = input_data = {
    'CreditScore': 600,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}

# Initialize OneHotEncoder
label_encoder_geo = OneHotEncoder()

# ✅ Fit the encoder on all possible categories (replace with your training categories)
label_encoder_geo.fit([['France'], ['Germany'], ['Spain']])

# Transform the input
geo_encoded = label_encoder_geo.transform([[input_data['Geography']]]).toarray()

# Convert to DataFrame with proper column names
geo_encoded_df = pd.DataFrame(
    geo_encoded,
    columns=label_encoder_geo.get_feature_names_out(['Geography'])
)

geo_encoded_df


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [ ]:
input_df=pd.DataFrame([input_data])
input_df


,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [ ]:
### ENCODE CATEGORICAL VARIABLES
from sklearn.preprocessing import LabelEncoder

# Create the encoder
label_encoder_gender = LabelEncoder()

# Fit and transform the Gender column
input_df['Gender'] = label_encoder_gender.fit_transform(input_df['Gender'])

# Show the updated DataFrame
input_df


,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,0,40,3,60000,2,1,1,50000


In [ ]:
## CONCATENATION OF ONE-HOT ENCODED COLUMNS

# Drop 'Geography' safely and concatenate
input_df = pd.concat(
    [input_df.drop("Geography", axis=1, errors="ignore"), geo_encoded_df],
    axis=1
)

input_df


,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,0,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [ ]:
input_scaled = scaler.transform(input_df)
input_scaled


array([[-0.53598516, -1.09499335,  0.10479359, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

In [ ]:
## predict churn
prediction=model.predict(input_scaled)
prediction

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 142ms/step


array([[0.08181855]], dtype=float32)

In [ ]:
prediction_proba=prediction[0][0]

In [ ]:
prediction_proba

np.float32(0.08181855)

In [ ]:
if prediction_proba>0.5:
    print('the customer is likely to churn.')
else:
    print('the customer isnot likely to churn')

the customer isnot likely to churn
